# 文章データへの前処理
今回のサンプルでは、下記のように文書への前処理を行います。<br>

- 後の扱いを考え、文字コードをUTF-8に統一（`Webスクレイピング時点で実施済み`）
- Pythonライブラリ BeautifulSoup を使用しhtmlタグを除去（`Webスクレイピング時点で実施済み`）
- 表記のゆらぎの修正　→　Pythonライブラリ neologdn を使用
- 形態素解析ソフト Janome を使用し、単語分割し不要単語を除去する

今回のターゲットは Naive Bayes での文書分類なので、名詞などの判定につながりそうな単語だけに絞ることがポイント。

In [1]:
## 必要なライブラリの読み込み
import sys
import re
import random
import glob
## 以下を使用するには事前準備が必要
import neologdn
from janome.tokenizer import Tokenizer

ModuleNotFoundError: No module named 'neologdn'

### 日本語文書の形態素解析の動作確認
janome の Tokenizer がどのような動作をするか確認します。
- node.surface → 分割した単語そのもの
- node.part_of_speech → node.surfaceに対応するcsv文字列: 品詞,品詞細分類1,分類2,分類3

品詞や細分類にどのような項目が入ってくるかは、「IPA品詞体系」を検索し参照してみて下さい。 →参考： [http://www.unixuser.org/~euske/doc/postag/](http://www.unixuser.org/~euske/doc/postag/)<br>
（**janome は分析結果をIPA品詞体系で返します**）<br><br>
neologdn の動作も確認します。

In [ ]:
nodelist = list(Tokenizer().tokenize('隣の客はよく柿食う客だ。'))
for node in nodelist:
    print('--------')
    for key, value in node.__dict__.items():
        print(key, ':', value)

In [ ]:
neologdn.normalize('わーい。わーーーい。')

### preprocess_text_1
与えられた文書の文字列（input_text）を前処理・単語分割し、必要な単語の羅列（List型）を返す関数を定義する。<br>
※ この処理はあくまで１例<br>
### preprocess_file_1
与えられたファイル（複数）の中身の文書テキストを preprocess_text_1 で前処理し、結果の単語の羅列（List型）をまとめてリストとして返す関数を定義

In [ ]:
def preprocess_text_1(input_text):
    input_text_reg = neologdn.normalize(input_text)  ### neologdn による表記ゆれ正規化 -> input_text_reg
    nodelist = Tokenizer().tokenize(input_text_reg)  ### input_text_reg を形態素解析 -> nodelist
    res = []                                         ### 結果の戻り値用入れ物
    ### 形態素解析した結果（単語）でループして処理する
    for node in nodelist:
        ### node.surface は 分割した単語そのもの　　例：「隣」
        sur = node.surface
        ### node.part_of_speech はsurfaceに対応するcsv文字列: 品詞,品詞細分類1,分類2,分類3
        a = node.part_of_speech.split(',')
        hin = a[0]           ### 品詞　　　　　　例：「名詞」
        bun1 = a[1]          ### 品詞細分類1　　 例：「一般」
        if hin == '動詞':    ### 動詞は原形(node.base_form)に直しておく
            sur = node.base_form
        ### まずは単語全部を対象とする
        taisho = True        
        ### 品詞が記号や助詞だったら対象から外す
        if hin in ['BOS/EOS','連体詞','接続詞','助詞','助動詞','連語','副詞','接頭詞','記号']:
            taisho = False
        ### 品詞細分類1が下記のものだったら対象から外す
        if bun1 in ['形容動詞語幹','副詞可能','代名詞','ナイ形容詞語幹','特殊','数','接尾','非自立']:
            taisho = False
        ### 名詞のうち、数字入りだったり、カッコが入ってしまっているものは対象から外す
        if hin == '名詞' and ( re.search('\d',sur) or re.search('[\(\)\（）]',sur) ):
            taisho = False
        ### 長さが1文字のものは判定用の単語とみなさない。対象から外す
        if len(sur) <= 1:
            taisho = False
        ### 特定の単語が出てきたら、対象から外す
        #if sur in ['判定には不適切な単語１','判定には不適切な単語２']:
        #    taisho = False
        
        # taisho = True  ### これは全部を対象にしてみる テスト用
        ### それでも対象として残ったものを出力する
        if taisho == True:
            res.append(sur)
    return res

def preprocess_file_1(input_files):
    res = []
    for input_file in input_files:
        with open(input_file, 'r', encoding='utf-8') as input_file_handle:
            res += preprocess_text_1( input_file_handle.read() )
    return res

Webスクレイピングで取得したファイルを前処理して、１行１単語の前処理済みテキストとして書き出す。<br>
- mlit_y28.txt , mlit_y29.txt の２つを前処理 → mlit.txt へ書き込む
- env_y28.txt , env_y29.txt の２つを前処理 → env.txt へ書き込む <br>
ここで作った mlit.txt と env.txt は学習の為のデータ（単語を数える）として使います。<br>
glob.globを使うと、存在するファイルのリストを作るのに便利です。又、List型を文字列として連結する際はjoinを使うのが便利です。

In [ ]:
words_m = preprocess_file_1(glob.glob('mlit_y2[89].txt'))
with open('mlit.txt', 'w', encoding='utf-8') as mlit_out_file:
    mlit_out_file.write( "\n".join( words_m ) + "\n" )

words_e = preprocess_file_1(glob.glob('env_y2[89].txt'))
with open('env.txt', 'w', encoding='utf-8') as env_out_file:
    env_out_file.write( "\n".join( words_e ) + "\n")

[glob.glob('mlit.txt'),glob.glob('env.txt')]    # ファイルが出来たか確認用

検証用データ（判定する文書）として使う為、mlit,envの両方で、<br>
30年度のテキストの中ランダムな場所から、全体の1%を抜き出しておきます。<br>
配列(List)の範囲指定を使って、切り出しをします。 （例） hairetsu[50:100] 

In [ ]:
test_ratio = 0.01      # = 1%

with open('mlit_check.txt', 'w', encoding='utf-8') as mlit_out_file:
    a = preprocess_file_1(['mlit_y30.txt'])
    r = random.random()
    mlit_out_file.write( ("\n".join(a[int(r * (1.0 - test_ratio)):][0:int(test_ratio * len(a))])) + "\n" )

with open('env_check.txt', 'w', encoding='utf-8') as env_out_file:
    a = preprocess_file_1(['env_y30.txt'])
    r = random.random()
    env_out_file.write( ("\n".join(a[int(r * (1.0 - test_ratio)):][0:int(test_ratio * len(a))])) + "\n" )

[glob.glob('mlit_check.txt'),glob.glob('env_check.txt')]    # ファイルが出来たか確認用

### うまくいかない場合
***ここはうまくいっていれば実行不要です***

ここまでうまくいっていれば、<br>
mlit.txt , mlit_check.txt , env.txt , env_check.txt の4つが作成されているはずです。<br>
開いて確認してみてください。<br>
必要なライブラリのインストールができなかった場合は、上記4つのファイルを<br>
既に準備してあるものの中から下記を実行してコピーします。

In [ ]:
import shutil,glob
target = ['mlit.txt','mlit_check.txt','env.txt','env_check.txt']
for f in target:
    shutil.copy('file/' + f,'./')
    print(glob.glob(f))  # ファイルが出来たことを確認